# Worksheet 08

Name:  Viktoria Zruttova
UID: U13977544

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [17]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[6.5149014364609865, 4.987564022066045, 4.187641641181017, 4.709033056053418, 5.0981848773142096, 7.485074853462232, 6.164500711586899, 4.808538414682599, 6.549479498974884, 6.058694263788755]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [18]:
s2 = np.random.normal( 8,1, 10 ).tolist()
print(s2)

[9.191251932784233, 6.997380116885031, 7.420603229717188, 7.570736521050409, 7.529131216909604, 9.393493354616833, 8.427069663095192, 7.148352348991734, 7.411130650581626, 7.567313595281277]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [19]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[6.058694263788755, 7.567313595281277, 6.549479498974884, 7.411130650581626, 4.808538414682599, 6.164500711586899, 7.485074853462232, 7.148352348991734, 8.427069663095192, 5.0981848773142096]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

We need a prior probability of being species 1 and a prior probability of being species 2, Then we need a mean and a variance for both species 1 and species 2. 

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [20]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s2) + len(s1)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , 
       sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2)  ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[7.567313595281277, 7.411130650581626, 7.485074853462232, 7.148352348991734, 8.427069663095192]
[6.058694263788755, 6.549479498974884, 4.808538414682599, 6.164500711586899, 5.0981848773142096]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 7.607788222282411,  mean_2 = 5.73587955326947
var_1 = 0.18753487355039233,  var_2 = 0.4432972786788314


/opt/homebrew/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [21]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

        
    # pdf_i[0] is probability p(p | s_0)
    
    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] *prob_s[0]/prob_x )
    prob_s1_x.append( pdf_i[1] *prob_s[1]/prob_x  )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  6.058694263788755
probability of observing that point if it came from cluster 0 =  3.245813531366332e-15
probability of observing that point if it came from cluster 1 =  0.6903402208680604
point =  7.567313595281277
probability of observing that point if it came from cluster 0 =  2.078324022079427
probability of observing that point if it came from cluster 1 =  0.00017695476285189073
point =  6.549479498974884
probability of observing that point if it came from cluster 0 =  2.5850261317992795e-07
probability of observing that point if it came from cluster 1 =  0.16701777947726892
point =  7.411130650581626
probability of observing that point if it came from cluster 0 =  1.2275537872282818
probability of observing that point if it came from cluster 1 =  0.000712945263981787
point =  4.808538414682599
probability of observing that point if it came from cluster 0 =  8.850582093344626e-49
probability of observing that point if it came from cluster 1 =  0.10091443051666653
point = 

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [22]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x),  sum(prob_s1_x)/ len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),  
        sum(prob_s1_x)/ len(prob_s1_x) ]
var = [sum([(x - mean[0]) ** 2 * prob for x, prob in zip(data, prob_s0_x)]) / sum(prob_s0_x),
       sum([(x - mean[1]) ** 2 * prob for x, prob in zip(data, prob_s1_x)]) / sum(prob_s1_x)]


print("P(S_1) = " + str(prob_c[0]) + ",  P(S_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.49486361358166303,  P(S_2) = 0.5051363864183369
mean_1 = 7.6124887087328315,  mean_2 = 0.5051363864183369
var_1 = 0.18729567674450703,  var_2 = 27.970925247948486


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [25]:
import numpy as np
from scipy.stats import norm


num_clusters = 2
data_points = [1.2, 2.3, 0.8, 3.1, 1.8, 2.5]  

# Initialize parameters
cluster_means = [1.0, 2.5]  
cluster_variances = [0.2, 0.3]  
cluster_prior_probabilities = [0.4, 0.6]  

updated_prob_cluster_0 = []  
updated_prob_cluster_1 = []  

for data_point in data_points:
    likelihoods = []

    for cluster_index in range(num_clusters):
       
        likelihoods.append(norm.pdf(data_point, cluster_means[cluster_index], 
                                    np.sqrt(cluster_variances[cluster_index])))

  
    marginal_likelihood = cluster_prior_probabilities[0] * likelihoods[0] 
    + cluster_prior_probabilities[1] * likelihoods[1]

  
    updated_prob_cluster_0.append(likelihoods[0] * cluster_prior_probabilities[0] / marginal_likelihood)
    updated_prob_cluster_1.append(likelihoods[1] * cluster_prior_probabilities[1] / marginal_likelihood)


for old_prob_0, old_prob_1, new_prob_0, new_prob_1 in zip(prob_s0_x, prob_s1_x, updated_prob_cluster_0, updated_prob_cluster_1):
    print(f"Original P(S_0 | X_i): {old_prob_0}, Updated P(S_0 | X_i): {new_prob_0}")
    print(f"Original P(S_1 | X_i): {old_prob_1}, Updated P(S_1 | X_i): {new_prob_1}")
    print()

Original P(S_0 | X_i): 4.701759267749032e-15, Updated P(S_0 | X_i): 1.0
Original P(S_1 | X_i): 0.9999999999999953, Updated P(S_1 | X_i): 0.08094914891608032

Original P(S_0 | X_i): 0.9999148642387838, Updated P(S_0 | X_i): 1.0
Original P(S_1 | X_i): 8.513576121610763e-05, Updated P(S_1 | X_i): 78.34059218602802

Original P(S_0 | X_i): 1.547752663763572e-06, Updated P(S_0 | X_i): 1.0
Original P(S_1 | X_i): 0.9999984522473363, Updated P(S_1 | X_i): 0.010955275996320474

Original P(S_0 | X_i): 0.999419551759303, Updated P(S_0 | X_i): 1.0
Original P(S_1 | X_i): 0.0005804482406969999, Updated P(S_1 | X_i): 41263.45638887837

Original P(S_0 | X_i): 8.77038303444908e-48, Updated P(S_0 | X_i): 1.0
Original P(S_1 | X_i): 1.0, Updated P(S_1 | X_i): 2.68066765202749

Original P(S_0 | X_i): 5.187907784963872e-13, Updated P(S_0 | X_i): 1.0
Original P(S_1 | X_i): 0.9999999999994812, Updated P(S_1 | X_i): 339.58780844873803



h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [26]:
cluster_assignments = []

for prob_0, prob_1 in zip(updated_prob_cluster_0, updated_prob_cluster_1):
    if prob_0 > prob_1:
        cluster_assignments.append(0)
    else:
        cluster_assignments.append(1)


for data_point, cluster_label in zip(data_points, cluster_assignments):
    print(f"Data point {data_point} is assigned to cluster {cluster_label}")

Data point 1.2 is assigned to cluster 0
Data point 2.3 is assigned to cluster 1
Data point 0.8 is assigned to cluster 0
Data point 3.1 is assigned to cluster 1
Data point 1.8 is assigned to cluster 1
Data point 2.5 is assigned to cluster 1
